# hub

In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import time
import gradio as gr
from skimage.metrics import structural_similarity as ssim

tf.get_logger().setLevel('ERROR')

# Load & Preprocess Images
def load_img(img_path, target_size=(1024, 1024)):  
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, target_size)
    return img.numpy()

# Calculate SSIM & PSNR
def calculate_metrics(original, generated):
    generated_resized = cv2.resize(generated, (original.shape[1], original.shape[0]))
    original_gray = cv2.cvtColor(original, cv2.COLOR_RGB2GRAY)
    generated_gray = cv2.cvtColor(generated_resized, cv2.COLOR_RGB2GRAY)
    ssim_score, _ = ssim(original_gray, generated_gray, full=True)
    psnr_score = cv2.PSNR(original, generated_resized)
    return ssim_score, psnr_score

# Compute Content & Style Loss
def calculate_content_loss(original, generated):
    original_gray = cv2.cvtColor(original, cv2.COLOR_RGB2GRAY)
    generated_gray = cv2.cvtColor(generated, cv2.COLOR_RGB2GRAY)
    return np.mean((original_gray - generated_gray) ** 2)  # Mean Squared Error

def calculate_style_loss(style, generated):
    style_gray = cv2.cvtColor(style, cv2.COLOR_RGB2GRAY)
    generated_gray = cv2.cvtColor(generated, cv2.COLOR_RGB2GRAY)
    return np.mean((style_gray - generated_gray) ** 2)  # Mean Squared Error

# Load Style Transfer Model
model_url = "https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2"
model = hub.load(model_url)

# Print Model Parameter Details
#print("Model Loaded. Parameter Count:")
#for var in model.variables:
#    print(f"Layer: {var.name}, Shape: {var.shape}")

# Function to Perform Style Transfer
def style_transfer(content_img, style_img):
    start_time = time.time()  # Start timer

    content = load_img(content_img)
    style = load_img(style_img)
    
    stylized_img = 0.6 * content + 0.5 * model(tf.expand_dims(content, 0), tf.expand_dims(style, 0))[0]
    stylized_img_np = (stylized_img.numpy().squeeze() * 255).astype(np.uint8)
    stylized_img_np = cv2.GaussianBlur(stylized_img_np, (5,5), 0)  

    end_time = time.time()  # End timer
    execution_time = end_time - start_time

    # Compute Metrics
    content_cv2 = cv2.imread(content_img)
    content_cv2 = cv2.resize(content_cv2, (1024, 1024))  # Match resolution
    content_cv2 = cv2.cvtColor(content_cv2, cv2.COLOR_BGR2RGB)

    style_cv2 = cv2.imread(style_img)
    style_cv2 = cv2.resize(style_cv2, (1024, 1024))
    style_cv2 = cv2.cvtColor(style_cv2, cv2.COLOR_BGR2RGB)

    ssim_score, psnr_score = calculate_metrics(content_cv2, stylized_img_np)
    content_loss = calculate_content_loss(content_cv2, stylized_img_np)
    style_loss = calculate_style_loss(style_cv2, stylized_img_np)

    return stylized_img_np, f"SSIM: {ssim_score:.4f} ,  PSNR: {psnr_score:.4f} ,  Content Loss: {content_loss:.4f} ,  Style Loss: {style_loss:.4f} ,  Execution Time: {execution_time:.2f}s"

# Gradio UI
interface = gr.Interface(
    fn=style_transfer,
    inputs=[
        gr.Image(type="filepath", label="Upload Content Image"),
        gr.Image(type="filepath", label="Upload Style Image")
    ],
    outputs=[
        gr.Image(label="Stylized Image"),
        gr.Textbox(label="Image Quality Metrics")
    ],
    title="Neural Style Transfer",
    description="Upload a content image and a style image to apply style transfer."
)

# Launch UI
interface.launch()


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
